# (Colab only) 구글 드라이브 DATA 확인
- 구글 드라이브에 업로드된 데이터 확인

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
# 데이터 목록 확인
!ls /content/drive/My\ Drive/Colab\ Notebooks/retrosynthesis/data

In [ ]:
import os

# 작업 디렉터리를 원하는 경로로 변경
current_dir = "/content/drive/My Drive/Colab Notebooks/retrosynthesis/"
os.chdir(current_dir)

# 변경된 현재 작업 디렉터리 확인
print("Current working directory:", os.getcwd())

### Colab GPU 사용 설정
- 런타임 - 런타임 유형 변경 -  GPU 선택

In [ ]:
# GPU 사용 가능 여부 확인
import torch
print("GPU 사용 가능 여부:", torch.cuda.is_available())
print("GPU 이름:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "사용 불가")

# Transformers 버전 확인

In [ ]:
import transformers
transformers.__version__

# RDkit 설치

In [ ]:
!pip install rdkit

# 테스트 정확도 확인

In [ ]:
import torch
from transformers import EncoderDecoderModel, PreTrainedTokenizerFast

from datautil import get_test_dataset, tokenize_function
from evaluate import topk_accuracy

In [ ]:
# 체크포인트 로딩
checkpoint = "./checkpoints/checkpoint-381555"  # 학습한 체크포인트 경로
model = EncoderDecoderModel.from_pretrained(checkpoint)
tokenizer = PreTrainedTokenizerFast.from_pretrained(checkpoint)

# 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Dataset 구성
raw_datasets = get_test_dataset()
tokenized_datasets = raw_datasets.map(
    lambda x: tokenize_function(x, tokenizer), batched=True)

In [ ]:
# top-k 정확도 계산
print("Calculating top-k accuracy...")
topk_accuracies = topk_accuracy(tokenized_datasets['test'],
                                model,
                                tokenizer,
                                batch_size=8)

# 결과 출력
for k in range(10):
    print(f"Top-{k+1} accuracy: {topk_accuracies[k]:.2f}%")


# 학습 모델을 활용한 역합성 예측

In [ ]:
input_smiles = "CC(=O)Oc1ccccc1C(=O)O"  # 예: 아스피린 SMILES

inputs = tokenizer(
    input_smiles,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=128
)

In [ ]:
# GPU 사용 시
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}

# generate 호출
outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=128,
    num_beams=10,           # 빔 서치 크기, 값이 크면 탐색 폭 증가
    early_stopping=True    # 빠른 종료
)

In [ ]:
predicted_smiles = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("예측된 반응물 SMILES:", predicted_smiles)

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw

reaction_smiles = input_smiles + '>>' + predicted_smiles.replace(' ', '')
print('retrosynthesis result:', reaction_smiles)

# 1. SMILES 문자열을 Reaction 객체로 변환
rxn = Chem.rdChemReactions.ReactionFromSmarts(reaction_smiles)

# 2. 반응 그림 생성
Draw.ReactionToImage(rxn)

In [ ]:
# 배치 입력의 경우

batch_inputs = ["CC(=O)Oc1ccccc1C(=O)O",
                "O=C(C)Oc1ccccc1C(=O)O",
                "N#Cc1ccc2[nH]ccc2c1"
               ]
inputs = tokenizer(batch_inputs,
                   return_tensors="pt",
                   padding=True,
                   truncation=True,
                   max_length=128)

inputs = {k: v.to(device) for k, v in inputs.items()}
outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=128,
    num_beams=10,
    early_stopping=True
)

predicted_outputs = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outputs]
print(predicted_outputs)

In [ ]:
src = batch_inputs[0]
tgt = predicted_outputs[0]

reaction_smiles = src + '>>' + tgt.replace(' ', '')
print('retrosynthesis result:', reaction_smiles)

# 1. SMILES 문자열을 Reaction 객체로 변환
rxn = Chem.rdChemReactions.ReactionFromSmarts(reaction_smiles)

# 2. 반응 그림 생성
Draw.ReactionToImage(rxn)